![image](images/um_logo.png)

# Computación II


# ***Análisis de ejecución y exclusión mutua***

## ***Lenguaje de máquina***
Cada arquitectura de procesador tiene su propio conjunto de instrucciones en lenguaje de máquina. Estas instrucciones son representadas por números almacenados como bytes en la memoria. Cada instrucción tiene un código único llamado opcode, que indica la operación a realizar. En el caso de las instrucciones del procesador 80x86, el tamaño puede variar. El opcode siempre se encuentra al inicio de la instrucción, y en muchos casos, las instrucciones también incluyen datos como constantes o direcciones que son utilizados por las mismas.

Programar directamente en lenguaje de máquina es muy complicado para los humanos, ya que descifrar el significado de las instrucciones numéricas resulta tedioso. Por ejemplo, la instrucción que suma los registros EAX y EBX y almacena el resultado en EAX se representa en código hexadecimal de la siguiente manera:

03 C3

Este código no resulta evidente para una persona. Afortunadamente, existe un programa llamado ensamblador que se encarga de realizar esta tarea aburrida por el programador.

## ***Lenguaje ensamblador***
Un programa escrito en lenguaje ensamblador se guarda como texto, al igual que los programas en lenguajes de alto nivel. Cada instrucción en el programa representa directamente una instrucción de la máquina. Por ejemplo, la instrucción de suma mencionada anteriormente se podría representar en lenguaje ensamblador de la siguiente manera:

add eax, ebx

En esta representación, el significado de la instrucción es mucho más claro que en el código de máquina. La palabra "add" es el mnemónico que representa la instrucción de suma. La forma general de una instrucción en lenguaje ensamblador es:

mnemónico operando(s)

Un ensamblador es un programa que lee un archivo de texto con instrucciones en lenguaje ensamblador y las convierte en código de máquina. Los compiladores realizan conversiones similares para lenguajes de programación de alto nivel. Sin embargo, un ensamblador es mucho más simple que un compilador.

Las instrucciones en un lenguaje de alto nivel suelen ser mucho más complejas y pueden requerir varias instrucciones de máquina. Otra diferencia importante entre los lenguajes ensamblador y de alto nivel es que cada tipo de CPU tiene su propio lenguaje de máquina y, por lo tanto, su propio lenguaje ensamblador. Transferir programas entre diferentes arquitecturas de computadoras es mucho más complicado en lenguaje ensamblador que en lenguajes de alto nivel.



>Fuente: Lenguaje Ensamblador para PC, Paul A. Carter, 9 de agosto de 2007 http://pacman128.github.io/pcasm/

## ***Análisis detallado de un problema de un problema de concurrencia (solución: "exclusión mutua")***

El siguiente ejemplo muestra un código en lenguaje ensamblador que lee dos números desde el teclado, los suma y muestra el resultado por pantalla.

In [ ]:
; file: first.asm
; First assembly program. This program asks for two integers as
; input and prints out their sum.
;
; To create executable:
; Using djgpp:
; nasm -f coff first.asm
; gcc -o first first.o driver.c asm_io.o
;
; Using Linux and gcc:
; nasm -f elf first.asm
; gcc -o first first.o driver.c asm_io.o
;
; Using Borland C/C++
; nasm -f obj first.asm
; bcc32 first.obj driver.c asm_io.obj
;
; Using MS C/C++
; nasm -f win32 first.asm
; cl first.obj driver.c asm_io.obj
;
; Using Open Watcom
; nasm -f obj first.asm
; wcl386 first.obj driver.c asm_io.obj

%include "asm_io.inc"
;
; initialized data is put in the .data segment
;
segment .data
;
; These labels refer to strings used for output
;
prompt1 db    "Enter a number: ", 0       ; don't forget nul terminator
prompt2 db    "Enter another number: ", 0
outmsg1 db    "You entered ", 0
outmsg2 db    " and ", 0
outmsg3 db    ", the sum of these is ", 0


;
; uninitialized data is put in the .bss segment
;
segment .bss
;
; These labels refer to double words used to store the inputs
;
input1  resd 1
input2  resd 1

 

;
; code is put in the .text segment
;
segment .text
        global  asm_main
asm_main:
        enter   0,0               ; setup routine
        pusha

        mov     eax, prompt1      ; print out prompt
        call    print_string

        call    read_int          ; read integer
        mov     [input1], eax     ; store into input1

        mov     eax, prompt2      ; print out prompt
        call    print_string

        call    read_int          ; read integer
        mov     [input2], eax     ; store into input2

        mov     eax, [input1]     ; eax = dword at input1
        add     eax, [input2]     ; eax += dword at input2
        mov     ebx, eax          ; ebx = eax
        dump_regs 1               ; dump out register values
        dump_mem 2, outmsg1, 1    ; dump out memory
;
; next print out result message as series of steps
;
        mov     eax, outmsg1
        call    print_string      ; print out first message
        mov     eax, [input1]     
        call    print_int         ; print out input1
        mov     eax, outmsg2
        call    print_string      ; print out second message
        mov     eax, [input2]
        call    print_int         ; print out input2
        mov     eax, outmsg3
        call    print_string      ; print out third message
        mov     eax, ebx
        call    print_int         ; print out sum (ebx)
        call    print_nl          ; print new-line

        popa
        mov     eax, 0            ; return back to C
        leave                     
        ret

### Go to Python!
En el siguiente fragmento de código dos hilos distintos realizan operaciones de suma y sesta sobre una misma variable global. 
Para poder ver la diferencia entre el resultado esperado y lo realmente obtenido se recomienda su ejecución en python 2.


In [4]:
import threading


# Variable compartida entre hilos
cont = 0
loop_amount = 100 #Incrementar la cantidad de incrementos y decrementos

# Función que incrementa el contador de forma segura
def add(loop_amount):
    global cont
    for i in range(loop_amount):
        cont = cont + 1

def sub(loop_amount):
    global cont
    for i in range(loop_amount):
        cont = cont - 1

# Creamos dos hilos que incrementan y decrementan el contador
t1 = threading.Thread(target=add, args=(loop_amount,))
t2 = threading.Thread(target=sub, args=(loop_amount,))
t1.start()
t2.start()

# Esperamos a que todos los hilos terminen
t1.join()
t2.join()


# Imprimimos el resultado final
print ("Resultado:\n", cont)


Resultado:
 0


El siguiente ejemplo escrito en C, muestra la misma tarea. Este programa en generál mostrará la diferencia entre, el valor esperado y el obtenido.

In [ ]:
/*
 ** Author: Carlos Taffernaberry <ctaffer@unsl.edu.ar> 
 **
 ** This program is free software; you can redistribute it and/or modify it
 ** under the terms of the GNU General Public License as published by the
 ** Free Software Foundation; either version 2 of the License, or (at your
 ** option) any later version.  See <http://www.fsf.org/copyleft/gpl.txt>.
 **
 ** This program is distributed in the hope that it will be useful, but
 ** WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
 ** or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
 ** for more details.
 **/
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>

int glob = 200;

void *hilo_suma()
{
    printf("Hello World! .... soy yo , el hilo_suma \n");
    int i;
    for (i = 0; i < 100000; i++) {
        glob++;
    }
    pthread_exit(NULL);
}

void *hilo_resta()
{
    printf("Hello World! .... soy yo , el hilo_resta \n");
    int i;
    for (i = 0; i < 100000; i++) {
        glob--;
    }
    pthread_exit(NULL);
}

int main(int argc, char *argv[])
{
    pthread_t hilo_s,hilo_r;
    int rc;
    printf("El main ... creando los threads\n");
    rc = pthread_create(&hilo_s, NULL, hilo_suma, NULL);
    if (rc) {
    printf("ERROR; pthread_create() = %d\n", rc);
    exit(-1);
    }
    rc = pthread_create(&hilo_r, NULL, hilo_resta, NULL);
    if (rc) {
    printf("ERROR; pthread_create() = %d\n", rc);
    exit(-1);
    }
   /* espera por finalización de TODOS*/
    pthread_join(hilo_s, NULL);
    pthread_join(hilo_r, NULL);
    printf ("valor global %d\n", glob);
    pthread_exit(NULL);
    return (0);
}

## ***Una solución***
Una posibilidad es usar una exclusión mutua para que cuando un hilo haga su operación tenga acceso exclusivo a la variable global. En el siguiente ejemplo se utiliza un llamado a time.sleep(...) a modo de simulación de una tarea cualquiera que el hilo debe realizar. Conceptualmente, esa tarea simulada, es responsabilidad solo del hilo y no tiene nada que compartir con otro hilo o proceso.

In [22]:
%%time
import threading
import time

# Variable compartida entre hilos
cont = 0
loop_amount = 100 #Incrementar la cantidad de incrementos y decrementos
lock = threading.Lock()

# Función que incrementa el contador de forma segura
def add(loop_amount):
    global cont
    lock.acquire()
    for i in range(loop_amount):
        time.sleep(0.1)
        cont = cont + 1
    lock.release()
        
def sub(loop_amount):
    global cont
    lock.acquire()
    for i in range(loop_amount):
        time.sleep(0.1)
        cont = cont - 1
    lock.release()

# Creamos dos hilos que incrementan y decrementan el contador
t1 = threading.Thread(target=add, args=(loop_amount,))
t2 = threading.Thread(target=sub, args=(loop_amount,))
t1.start()
t2.start()

# Esperamos a que todos los hilos terminen
t1.join()
t2.join()


# Imprimimos el resultado final
print ("Resultado:\n", cont)

Resultado:
 0
Tiempo de ejecución:  20.03770923614502
CPU times: user 11.1 ms, sys: 8.25 ms, total: 19.3 ms
Wall time: 20 s


El problema con la aproximación anterior es que la tarea propia del hilo está dentro de lo que hemos evaluado como sección crítica del código. Esto es un error conceptual ya que de resolverlo de esa forma no estaríamos aprovechando la concurrencia.
En el siguiente ejemplo se considera mejor cuál es exactamente la sección crítica, hubicándose los llamados a las primitivas **acquire()** y **release()** de forma tal que permita realizar de forma concurrente las tareas que son propias y aisladas de cada hilo. 

In [24]:
%%time
import threading
import time

# Variable compartida entre hilos
cont = 0
loop_amount = 100 #Incrementar la cantidad de incrementos y decrementos
lock = threading.Lock()

# Función que incrementa el contador de forma segura
def add(loop_amount):
    global cont
    for i in range(loop_amount):
        time.sleep(0.1)
        lock.acquire()
        cont = cont + 1
        lock.release()
        
def sub(loop_amount):
    global cont
    for i in range(loop_amount):
        time.sleep(0.1)
        lock.acquire()
        cont = cont - 1
        lock.release()

# Creamos dos hilos que incrementan y decrementan el contador
t1 = threading.Thread(target=add, args=(loop_amount,))
t2 = threading.Thread(target=sub, args=(loop_amount,))
t1.start()
t2.start()

# Esperamos a que todos los hilos terminen
t1.join()
t2.join()


# Imprimimos el resultado final
print ("Resultado:\n", cont)


Resultado:
 0
CPU times: user 13.7 ms, sys: 6.09 ms, total: 19.7 ms
Wall time: 10 s
